# NLP (comentários página do Facebook)
Este notebook contém o código referente ao modelo que analisa sentimentos em formato textual. A proposta aqui foi a de criar um modelo que pudesse analisar os comentários públicos da página do Facebook da Passos Mágicos e assim demonstrar o quão impactante suas ações são nas vidas das pessoas. Foram considerados 3 categorias de comentários: positivo, negativo e neutro.

Aqui vale notar que além dos comentários da página do Facebook da ONG, também foram gerados batches de comentários pelo ChatGPT da OpenAI. Isso se fez necessário, pois não haviam muitos comentários ou reviews da ONG que pudessem ser utilizados para treinar efetivamente um NLP.

Os comentários e reviews originais da página da ONG foram misturados juntos dos comentários gerados pelo ChatGPT.

In [3]:
import pandas as pd
import nltk
import re
import string
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn import svm
import joblib

In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Funções utilitárias para limpeza e normalização de strings

In [5]:
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text)
    return " ".join([w for w in text.split()])

def tokenizer(text):
    stop_words = stopwords.words("portuguese")
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) > 2]
        return " ".join([t for t in text])
    else:
        return None

## Carregando os comentários e reviews da página do Facebook da ONG Passos Mágicos + comentários gerados pelo ChatGPT da OpenAI

In [6]:
df_1 = pd.read_csv('/content/chatgpt-base-comentarios-negativos.csv', sep=';')
df_2 = pd.read_csv('/content/chatgpt-base-comentarios-neutros.csv', sep=';')
df_3 = pd.read_csv('/content/chatgpt-base-comentarios-positivos.csv', sep=';')
df_4 = pd.read_csv('/content/facebook-mentions.csv', sep=';')
df_5 = pd.read_csv('/content/facebook-reviews.csv', sep=';')
df = pd.concat([df_1, df_2, df_3, df_4, df_5])
df

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo
...,...,...
7,De projeto...uma ONG! Exemplo para muitas pess...,positivo
8,Projeto sério realizado por uma equipe extrema...,positivo
9,“A educação é a arma mais poderosa que você po...,positivo
10,UM exemplo de instituição... estão de parabéns...,positivo


In [7]:
df['categoria'].unique()

array(['negativo', 'neutro', 'positivo'], dtype=object)

In [8]:
df.shape

(1060, 2)

In [9]:
df.head()

,texto,categoria
0,O atendimento foi péssimo.,negativo
1,Produto de baixa qualidade.,negativo
2,Muito insatisfeito com a compra.,negativo
3,Demorou muito para chegar.,negativo
4,Chegou quebrado e não funciona.,negativo


In [10]:
df = shuffle(df)
df = df.reset_index(drop=True)

In [11]:
df.head()

,texto,categoria
0,"A ONG oferece boas oportunidades, mas a estrut...",neutro
1,"A ONG realiza um trabalho significativo, no en...",neutro
2,Serviço excelente.,positivo
3,Não cumpriram o prometido.,negativo
4,Atendimento adequado.,neutro


In [12]:
df['texto_processado'] = df['texto'].apply(tokenizer)
df.head()

,texto,categoria,texto_processado
0,"A ONG oferece boas oportunidades, mas a estrut...",neutro,ong oferece boas oportunidades estrutura apoio...
1,"A ONG realiza um trabalho significativo, no en...",neutro,ong realiza trabalho significativo entanto acr...
2,Serviço excelente.,positivo,servico excelente
3,Não cumpriram o prometido.,negativo,nao cumpriram prometido
4,Atendimento adequado.,neutro,atendimento adequado


## Dividindo o dataset em treino e teste

In [13]:
X = df['texto_processado'].values
y = df['categoria'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=777, stratify=y)

In [14]:
print('X_train:', X_train.size)
print('X_test:', X_test.size)
print('y_train:', y_train.size)
print('y_test:', y_test.size)

X_train: 742
X_test: 318
y_train: 742
y_test: 318


In [15]:
vect = CountVectorizer(lowercase=False)
vect.fit(X_train)

CountVectorizer(lowercase=False)

In [16]:
X_train = vect.transform(X_train)
X_train

<742x684 sparse matrix of type '<class 'numpy.int64'>'
	with 3824 stored elements in Compressed Sparse Row format>

In [17]:
X_test = vect.transform(X_test)
X_test

<318x684 sparse matrix of type '<class 'numpy.int64'>'
	with 1390 stored elements in Compressed Sparse Row format>

## Utilização do modelo SVM para categorizar as frases

In [18]:
modelo = svm.SVC(kernel='linear')
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

# TODO: talvez utilizar um xgboost? ou rodar o cross_val para validar varios modelos em paralelo e plotar o score deles

In [19]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7421383647798742


## Teste do modelo

In [20]:
texto_teste = tokenizer('Não gostei da ONG, não prestaram o serviço de forma adequada.')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['negativo'], dtype=object)

In [21]:
texto_teste = tokenizer('Gostei da ONG!')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['positivo'], dtype=object)

In [22]:
texto_teste = tokenizer('A OGN poderia oferecer mais serviços, mas ok.')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['neutro'], dtype=object)

In [23]:
texto_teste = tokenizer('Parece uma boa ONG, contudo poderia ser mais transparente!')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['neutro'], dtype=object)

## Exportando o modelo para uso no Streamlit

In [24]:
arquivo = "nlp_model.pkl"
joblib.dump(modelo, arquivo)

['nlp_model.pkl']

## Importando o modelo para teste

In [25]:
modelo = joblib.load(arquivo)

In [26]:
texto_teste = tokenizer('Parece uma boa ONG, contudo poderia ser mais transparente!')
text_vect = vect.transform([texto_teste])
modelo.predict(text_vect)

array(['neutro'], dtype=object)